## Import libraries

In [1]:
import src.config as config

2025-06-25 13:08:08.949 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 9
    # -----------------------------------------
    )

2025-06-25 13:08:12.455 | INFO     | src.dataset:main:33 - Starting raw data loading...
2025-06-25 13:08:12.455 | INFO     | src.dataset:main:39 - Requesting information between 2016-06-27 and 2025-06-25
2025-06-25 13:08:12.455 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-25 13:08:15.875 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-25 13:08:18.529 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-06-25 13:08:26.866 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading ACUCAR (SB=F) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-25 13:08:29.529 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-06-25 13:08:48.143 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-06-25 13:08:50.356 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-06-25 13:09:10.349 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-06-25 13:09:40.755 | WARNING  | src.utils.dataset.dataset_loading_strategy:_request_bcb_series:118 - Erro ao conectar à API do BCB: Expecting value: line 1 column 1 (char 0)
2025-06-25 13:09:40.758 | WARNING  | src.utils.dataset.dataset_loading_strategy:_load_single_ticker:94 - No data returned for SELIC_Meta_Anual (432)
2025-06-25 13:09:42.760 | INFO     | src.utils

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')
c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
INFO:src.utils.dataset.clean_strategy:Columns removed by f_classif: ['Volume_^GSPC']
c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (

            Close_^BVSP  High_^BVSP  Low_^BVSP  Open_^BVSP  Volume_^BVSP  \
2025-06-18     138717.0    139161.0   138443.0    138844.0     8323400.0   
2025-06-19          NaN         NaN        NaN         NaN           NaN   
2025-06-20     137116.0    138719.0   136815.0    138715.0    11289300.0   
2025-06-21          NaN         NaN        NaN         NaN           NaN   
2025-06-22          NaN         NaN        NaN         NaN           NaN   
2025-06-23     136551.0    137130.0   135835.0    137116.0     7727800.0   
2025-06-24     137165.0    138156.0   136254.0    136552.0     8083700.0   

            Close_^GSPC   High_^GSPC    Low_^GSPC   Open_^GSPC  Volume_^GSPC  \
2025-06-18  5980.870117  6018.250000  5971.890137  5987.930176  5.106470e+09   
2025-06-19          NaN          NaN          NaN          NaN           NaN   
2025-06-20  5967.839844  6018.200195  5952.560059  5999.669922  7.451500e+09   
2025-06-21          NaN          NaN          NaN          NaN         

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [1]:
from src import config
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    dataset_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    targets = ["^BVSP"],
    train_size_ratio = 1,
    batch_size = 1,
    sequence_length = 50
    # -----------------------------------------
)

2025-06-25 22:40:47.794 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


2025-06-25 22:40:56.323 | INFO     | src.features:main:36 - Generating features from dataset...
2025-06-25 22:40:56.356 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-25 22:40:56.371 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-25 22:40:56.457 | SUCCESS  | src.features:main:51 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-25 22:40:56.528 | SUCCESS  | src.features:main:55 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-25 22:40:56.530 | SUCCESS  | src.features:main:59 - Features generation complete.
2025-06-25 22:40:56.531 | INFO     | src.features:main:61 - Saving transformers...
2025-06-25 22:40:56.532 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-25 22:40:56.541 | INFO     | src.features:main:77 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [ ]:
from src import config
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy",
    # -----------------------------------------
    optimizer =  None,  # Use default optimizer
    loss = None,  # Use default loss
    metrics = None,  # Use default metrics
    # -----------------------------------------
    batch_size = 32,
    epochs = 1000,
    validation_len = 30,
    # -----------------------------------------
    experiment_name = "default_experiment",
)

2025-06-25 23:06:18.326 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


2025-06-25 23:06:29.216 | INFO     | src.modeling.train:main:42 - Loading training dataset...
2025-06-25 23:06:29.228 | INFO     | src.modeling.train:main:50 - Building model...
2025-06-25 23:06:29.228 | INFO     | src.modeling.train:main:56 - Selecting compile strategy...
2025-06-25 23:06:29.244 | INFO     | src.modeling.train:main:64 - Selecting training strategy...
2025-06-25 23:06:29.244 | INFO     | src.modeling.train:main:73 - Building model training pipeline template...
2025-06-25 23:06:29.244 | INFO     | src.modeling.train:main:80 - Training model...
Epoch 1/1000
68/68 ━━━━━━━━━━━━━━━━━━━━ 22s 123ms/step - loss: 0.6882 - mae: 1.0274 - mse: 53.7448 - r2_score: -0.0128 - rmse: 2.1740 - smape: 158.9926 - val_loss: 0.0838 - val_mae: 0.2135 - val_mse: 0.1173 - val_r2_score: -0.0053 - val_rmse: 0.3425 - val_smape: 118.7145
Epoch 2/1000
68/68 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - loss: 0.6990 - mae: 1.0449 - mse: 53.7642 - r2_score: -0.0287 - rmse: 2.1843 - smape: 179.7507 - val_loss: 

## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [3]:
from src import config
from src.modeling import predict

predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch47_loss0.2119.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    length = 50
    # -----------------------------------------
)

2025-06-25 13:28:53.264 | INFO     | src.modeling.predict:main:29 - Performing inference for model...
2025-06-25 13:28:53.296 | INFO     | src.modeling.predict:main:33 - Input data shape: (51, 15)
2025-06-25 13:28:53.319 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
            Close_^BVSP       type
2025-06-23    -0.004121       True
2025-06-24     0.004496       True
2025-06-25    -0.000232  Predicted
2025-06-25 13:28:55.044 | SUCCESS  | src.modeling.predict:main:76 - Inference complete.


## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.